In [1]:
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
folder = "env"
sourcedir = currentdir.split(folder)[0]
sys.path.insert(0, sourcedir)
# from env.equation.equation import Equation
print(sourcedir)

/home/valdecar/Documents/projects/project/parser/


### Добавление нового терма скобочного терма
Индексация: `a[i,j,] = a[j,i,] `

Чтобы добавить новый скобочный терм в Equation нужно:</br>
> 1) Добавить input pattern в data/terms/input/wolfram/patterns

> 2) Добавить правила грамматики в data/grammars

> 3) Добавить новые термы в tranlator/tree/nodes

> 4) Добавить терм в списки переменных terms_vars, terms_br: args/args_main

> 5) Добавить slambda pattern в data/terms/slambda/sympy/patterns/brackets


##### 1) Добавление input pattern

Для этого создается класс в data/terms/input/wolfram/patterns в методе `__call__` которого должен возвращаться re pattern:<br>
```

    def __init__(self, net):
        self.net = net
        self.id = 'idx'
        self.init_pattern()

    def init_pattern(self):
        
        # find 1 or 1,1, or 11, 123, 1,:
        self.idx = "(?P<idx>((\d)+|((\d)+,)+))"

        # find a, aa, Aa:
        self.obj = "(?P<obj>\w+)"

        # find a[1,2,3,]:
        self.main = r"%s\[" % (self.obj) 
        
        self.gen = lambda: self.main


    def __call__(self):
        return(self.gen())
```

После создания patterna необходимо добавить его в три списка файла `data/terms/input/wolfram/lex_net_wolfram.py`: 
> terms_gens - для tokenizera:<br>
>>        terms_gens = [Base, ArgInt, ArgFloat, ArgDelay, ArgTime,
                      Var, VarBdp, Coeffs, Bdp, Diff, Pow, Func,
                      FreeVar, Time, DiffTimeVar, Dot, Idx]


> patterns_order - для приоритета терма с другими   
>>        patterns_order = ['diff',
                          'bdp',
                          'dot',
                          'func',
                          'idx',
                          'diff_time',
                          'var',
                          'free_var',
                          'time',
                          'coeffs',
                          'pow',
                          'float']

> map_ptg - отображения lex терма в терм грамматики
>>         map_ptg = dict([('diff', 'a'),
                        ('bdp', 'a'),
                        ('diff_time', 'a'),
                        ('var', 'a'),
                        ('free_var', 'a'),
                        ('time', 'a'),
                        ('coeffs', 'a'),
                        ('pow', 'w'),
                        ('func', 'f'),
                        ('float', 'a'),
                        ('dot', 'a'),
                         ('idx', 'i')])



##### 2) Добавить правила грамматики 

Для этого создаются правила для скобок a[,] -> i,] в  data/grammars/gm_pow_f_args.py, согласованные с значением map_ptg['idx'] добавленным на предыдущем шаге:

                 ('T', ('T', 'M', 'I')), ('T', ('I')),
                 ('I', ('LI', 'E', 'RI')),
                 ('V', ('LI', 'AI', 'RI')),
                 ('AI', ('E', 'D', 'AI')), ('AI', ('E', 'D')),
                 ('LI', ('i')), ('RI', (']'))
                 ('D', ',')


##### 3) Добавить новые термы в tranlator/tree/nodes

Необходимо для корректного преобразования дерева разбора в дерево операций.
В `self.brs` класса NodeR нужно добавить новые скобочные термы грамматики:
>            `self.brs= ['(', ')', 'w', 'f', 'i', ']']`

После этого уже можно построить дерево операций:

In [11]:
from env.equation.equation import Equation
eq = Equation("a[i,j,] = a[j,i,]")
eq.parser.parse()

print("from lex:")
print(eq.parser.eq)

print("\ntree:")
print(eq.eq_tree)


from lex:
[['i', 'a', ',', 'a', ',', ']'], ['='], ['i', 'a', ',', 'a', ',', ']']]

tree:
=
child 0: br
   child 0: i
   child 1: args
      child 0: a
      child 1: a
   child 2: ]
child 1: br
   child 0: i
   child 1: args
      child 0: a
      child 1: a
   child 2: ]


##### 4) Добавить терм в списки переменных terms_vars, terms_br: args/args_main:

Этот шаг необходим для интерпретирования левой скобки (`a[`) как переменной со значением `a`
Нужно добавтить в два списка:
>               terms_vars = ['free_var', 'var', 'func', 'coeffs', 'idx']
              terms_br = ['func', 'idx']
  
Теперь можно производить подстановки:

In [13]:
import sympy
a = sympy.Matrix([[0, 1], [1, 0]])

eq.args_editor.get_vars()
eq.args_editor.subs(a=a, i=0, j=1)
    

##### 5) Добавить slambda pattern в data/terms/slambda/sympy/patterns/brackets

Нужно создать replacer для sympy семантики. Для этого создается класс Idx c `self.id` для поиска в списке термов:
> self.id = 'l:i|r:]'

Это означает, что левый скобочный терм i, правый ].

Основная часть класса заключается в строчках
>         self.func_name = left_node.args['variable']['value']
        return(lambda *A, func=self.func_name: func.__getitem__(A))

Это означает, что терм использует его значение для применения стандартного метода `__getitem__`. При этом количество индексов является произвольным (`*A`).
Использование значения по умолчанию (`func=self.func_name`) необходимо для исключения переполнения одного значения для всех `idx` термов (т.к. в настоящей реализации один объект класса Idx используется для генерации значения всех `idx` термов в уравнении)<br><br>

Теперь терм `a[i,j,]` имеет lambda представление т.е. его можно вызывать: 

In [16]:

eq.slambda.sympy.lambdify_sem()
out = eq.slambda.sympy.lambdify()

print("for matrix:")
print(a)
print("\n")
print(eq.sent)

print("\nlambdify:")
print(out())


for matrix:
Matrix([[0, 1], [1, 0]])


a[i,j,]=a[j,i,]

lambdify:
True
